In [1]:
import math
import numpy as np
from vpython import *
scence = canvas()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
"""Useful functions"""
# convert vpython vector to numpy array
# moving in a plain
def point(pos):
    return np.array([pos.x, pos.y])

# a penalty field for collision avoidance
def field_func(dist_obj, threshold):
    # return zero when d is greater than the threshold
    if dist_obj > threshold:
        return 0
    else:
        return math.log(threshold / dist_obj)

# sum of penalty
def sum_field(pos, list_objects, threshold):
    # total penalty of all the objects
    total = 0
    for temp_obj in list_objects:
        # compute the distance from pos to temp_object
        pos_to_obj = dist(pos, point(temp_obj.pos))
        # exclude the same object
        if pos_to_obj != 0:
            total = total + field_func(pos_to_obj, threshold)
    
    # return the sum
    return total

# distance function: compute the distance between two points
def dist(pointA, pointB):
    pointA_to_pointB = np.linalg.norm(pointA - pointB)
    return pointA_to_pointB

# cost function
def cost(current_pos, goal_pos, penalty):
    current_to_goal = dist(current_pos, goal_pos)
    return current_to_goal + penalty

# find the min cost locally
def cost_gradient(current_pos, goal_pos, delta_x, delta_y, list_objects, threshold):
    # four costs
    # for delta_x
    # x1 is x + delta_x, x2 is x - delta_x
    pos_x1 = np.array(current_pos)
    pos_x1[0] = pos_x1[0] + delta_x
    penalty_x1 = sum_field(pos_x1, list_objects, threshold)
    cost_x1 = cost(pos_x1, goal_pos, penalty_x1)
    
    pos_x2 = np.array(current_pos)
    pos_x2[0] = pos_x2[0] - delta_x
    penalty_x2 = sum_field(pos_x2, list_objects, threshold)
    cost_x2 = cost(pos_x2, goal_pos, penalty_x2)
    
    # for delta_y
    # y1 is y + delta_y, y2 is y - delta_y
    pos_y1 = np.array(current_pos)
    pos_y1[1] = pos_y1[1] + delta_y
    penalty_y1 = sum_field(pos_y1, list_objects, threshold)
    cost_y1 = cost(pos_y1, goal_pos, penalty_y1)
    
    pos_y2 = np.array(current_pos)
    pos_y2[1] = pos_y2[1] - delta_y
    penalty_y2 = sum_field(pos_y2, list_objects, threshold)
    cost_y2 = cost(pos_y2, goal_pos, penalty_y2)
    
    # compute the gradient
    delta_cost_x = cost_x1 - cost_x2
    delta_cost_y = cost_y1 - cost_y2
    gradient = np.array([delta_cost_x / 2*delta_x, delta_cost_y / 2*delta_y])
    # normalize
    gradient_len = math.sqrt(gradient[0]**2 + gradient[1]**2)
    
    return gradient / gradient_len

# =============================== Function End ===================================== #

# =============================== Main Program ===================================== #
"""Board: the ground that all the objects will be put on it"""
length = 200
width = 200
thickness = 0.2
board = box(pos=vec(0, 0, -thickness), size=vec(length, width, thickness), color=color.gray(0.8))

# used to stored all the objects on the board
list_obj = []

"""4 Obstacles (cylinders): in the middle of the board"""
offset = 15
obs_radius = 5
height = 15
orange = vec(1, 0.5, 0)
obs1 = cylinder(pos=vec(offset, offset, 0), axis=vec(0, 0, height), radius=obs_radius, color=orange)
list_obj.append(obs1)
obs2 = cylinder(pos=vec(-offset, offset, 0), axis=vec(0, 0, height), radius=obs_radius, color=orange)
list_obj.append(obs2)
obs3 = cylinder(pos=vec(-offset, -offset, 0), axis=vec(0, 0, height), radius=obs_radius, color=orange)
list_obj.append(obs3)
obs4 = cylinder(pos=vec(offset, -offset, 0), axis=vec(0, 0, height), radius=obs_radius, color=orange)
list_obj.append(obs4)

"""16 pedestrians: pedestrian and the velocity"""
list_ped = []
ped_radius = 3
position_ped = [vec(-95,  15, ped_radius), # ped 1
                vec(-95,   5, ped_radius), # ped 2
                vec(-95,  -5, ped_radius), # ped 3
                vec(-95, -15, ped_radius), # ped 4
                
                vec( 95,  15, ped_radius), # ped 5
                vec( 95,   5, ped_radius), # ped 6
                vec( 95,  -5, ped_radius), # ped 7
                vec( 95, -15, ped_radius), # ped 8
                
                vec( 15,  95, ped_radius), # ped 9
                vec(  5,  95, ped_radius), # ped 10
                vec( -5,  95, ped_radius), # ped 11
                vec(-15,  95, ped_radius), # ped 12
                
                vec( 15, -95, ped_radius), # ped 13
                vec(  5, -95, ped_radius), # ped 14
                vec( -5, -95, ped_radius), # ped 15
                vec(-15, -95, ped_radius)] # ped 16

v_scalar = 25   # Scalar of the velocity
list_varr = []  # list of velocity arrows
list_goal = []  # list of goal position
for i in range(16):
    ped = sphere(pos=position_ped[i], radius=ped_radius, color=color.green)
    list_obj.append(ped)
    list_ped.append(ped)
    # set initial velocity and goal pos
    temp_velocity = 0
    goal_pos = np.array([])
    if i <= 3:
        temp_velocity = vector(v_scalar, 0, 0)   # pedestrian move from left to right
        goal_pos = np.array([ped.pos.x + 200, ped.pos.y])
    elif i <= 7:
        temp_velocity = vector(-v_scalar, 0, 0)  # pedestrian move from right to left
        goal_pos = np.array([ped.pos.x - 200, ped.pos.y])
    elif i <= 11:
        temp_velocity = vector(0, -v_scalar, 0)  # pedestrian move from top to bottom
        goal_pos = np.array([ped.pos.x, ped.pos.y - 200])
    else:
        temp_velocity = vector(0, v_scalar, 0)   # pedestrian move from bottom to top
        goal_pos = np.array([ped.pos.x, ped.pos.y + 200])
        
    ped.velocity = temp_velocity
    list_goal.append(goal_pos)
    # initial velocity arrow
    v_arrow = arrow(pos=ped.pos, axis=0.2 * ped.velocity, color=color.yellow)
    list_varr.append(v_arrow)

# ped1 = sphere(pos=vec(-95, 15, ped_radius), radius=ped_radius, make_trail=True, color=color.green)
# list_obj.append(ped1)
# list_ped.append(ped1)
# initial velocity of each pedestrian
# ped1.velocity = vector(25, 0, 0)
# ped2.velocity = ped1.velocity
# arrow_scale = 1
# v_arrow1 = arrow(pos=ped1.pos, axis=arrow_scale * ped1.velocity, color=color.yellow)
# v_arrow2 = arrow(pos=ped2.pos, axis=arrow_scale * ped2.velocity, color=color.yellow)

# loop
threshold = 10000
num_ped = len(list_ped)
scene.autoscale = False

while True:
    rate(50)
    for i in range(num_ped):
        # get the corresponding object
        ped1 = list_ped[i]
        goal_pos = list_goal[i]
        v_arrow1 = list_varr[i]
        
        # computation for ped
        current_pos = np.array([ped1.pos.x, ped1.pos.y])
        ped1_gradient = cost_gradient(current_pos, goal_pos, 0.5, 0.5, list_obj, threshold)
        ped1.pos.x = ped1.pos.x - ped1_gradient[0]
        ped1.pos.y = ped1.pos.y - ped1_gradient[1]
        # computation for velocity
        ped1.velocity = vector(-25*ped1_gradient[0], -25*ped1_gradient[1], 0)
        v_arrow1.pos = ped1.pos
        v_arrow1.axis = ped1.velocity


In [16]:
a = np.array([1, 2, 3])
b = np.array(a)
b[1] = 1000
print (b)
print (a)

[   1 1000    3]
[1 2 3]
